### Imports

In [ ]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

### setting train and test directories

In [ ]:
train_dir = '../input/dog-breed-identification/train'
test_dir = '../input/dog-breed-identification/test'

train_size = len(os.listdir('../input/dog-breed-identification/train'))
test_size = len(os.listdir('../input/dog-breed-identification/test'))

train_size,test_size

In [ ]:
df = pd.read_csv('../input/dog-breed-identification/labels.csv')
df.head()

In [ ]:
#Extracting different classes

dog_breeds = sorted(df['breed'].unique())
n_classes = len(dog_breeds)
print(n_classes)
#dog_breeds

In [ ]:
#Converting classes to numbers
class_to_num = dict(zip(dog_breeds,range(n_classes)))

In [ ]:
#Function to load and convert images to array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical

def images_to_array(data_dir,df,image_size):
    image_names = df['id']
    image_labels = df['breed']
    data_size = len(image_names)
    
    X = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype = np.uint8)
    y = np.zeros([data_size,1],dtype = np.uint8)
    
    for i in range(data_size):
        img_name = image_names[i]
        img_dir = os.path.join(data_dir,img_name+'.jpg')
        img_pixels = load_img(img_dir,target_size=image_size)
        X[i] = img_pixels
        y[i] = class_to_num[image_labels[i]]
        
    y = to_categorical(y)
    
    print('Output Data Size: ', X.shape)
    print('Output Label Size: ', y.shape)
    return X, y     

In [ ]:
#Selecting image size according to pretrained models
img_size = (300,300,3)

In [ ]:
X, y = images_to_array(train_dir,df,img_size)

### Callbacks

In [ ]:
#Callbacks
from keras.callbacks import EarlyStopping
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
my_callback=[EarlyStop_callback]

### Model

In [ ]:
from keras import Sequential, regularizers
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

model = Sequential()

model.add(Conv2D(
    64, 3, 
    padding="same", 
    activation="relu", 
    input_shape=img_size
))
model.add(MaxPool2D())

model.add(Conv2D(
    64, 3, 
    padding="same", 
    activation="relu"
))
model.add(MaxPool2D())
model.add(Dropout(.3))

model.add(Flatten())
model.add(Dense(240, activation="relu"))
model.add(Dense(120, activation="softmax"))

In [ ]:
from keras.optimizers import Adam
#Compiling Model
model.compile(optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
from keras.utils import plot_model
plot_model(model, show_shapes=True)

In [ ]:
#Training Model
history = model.fit(X,
                  y,
                  batch_size=16,
                  epochs=100,
                  validation_split=.3,
                  callbacks=my_callback)

In [ ]:
import pandas as pd

# convert the training history to a dataframe
history_df = pd.DataFrame(history.history)
# use Pandas native plot method
history_df['loss'].plot();

In [ ]:
y_pred = model.predict(X, batch_size=16)
y_pred

In [ ]:
breed_test_pred = [dog_breeds[np.argmax(y_pred[k])] for k in range(len(y_pred))]


In [ ]:
del X, y, y_pred, df

# submission


In [ ]:
sample_df = pd.read_csv("../input/dog-breed-identification/sample_submission.csv")

In [ ]:
from tqdm import tqdm # show progress bar of loop
from keras.preprocessing.image import load_img

def images_to_array2(data_dir, labels_dataframe, img_size = (224,224,3)):
    '''
    Do same as images_to_array but omit some unnecessary steps for test data.
    '''
    images_names = labels_dataframe['id']
    data_size = len(images_names)
    X = np.zeros([data_size, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    for i in tqdm(range(data_size)):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name+'.jpg')
        img_pixels = load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
    print('Ouptut Data Size: ', X.shape)
    return X

test_data = images_to_array2(test_dir, sample_df, img_size)

In [ ]:
y_pred = model.predict(test_data, batch_size=16)

In [ ]:
#Create submission file
for b in dog_breeds:
    sub_df[b] = y_pred[:,class_to_num[b]]
sub_df.to_csv('pred.csv', index=None)